In [1]:

import sys
from fran.inference.inference_raytune_models import BoundingBoxes_to_lists, ModelFromTuneTrial
from fran.transforms.spatialtransforms import *
from fran.managers.learner_plus import *
from fran.managers.raytune_manager import *
from fran.inference.inference_base import *
from fran.inference.inference_neptune_models import *
from fran.utils.imageviewers import *
from fran.managers.raytune_manager import *
from fran.utils.helpers import *

/home/ub/Dropbox/code/fran/nbs


/home/ub/programs/miniconda3/lib/python3.9/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


https://app.neptune.ai/drusmanbashir/KiTS19/
Remember to stop your project once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/project#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


In [3]:
sys.path+= [".."]
import os; 
print(os.getcwd())

In [4]:
#For myself
if is_notebook() == False:
    os.chdir("nbs")
common_paths_filename="config.yaml"
proj_defaults = load_proj_defaults(common_paths_filename,"kits21")
config_dict =  load_config_from_workbook(proj_defaults.configuration_filename, raytune=False)

common_paths_filename="config.yaml"
proj_defaults = load_proj_defaults(common_paths_filename,"kits21")

# Processing KiTS21 test set

In [ ]:

train_list,valid_list,test_list = get_train_valid_test_lists_from_json(fold=0, json_fname=proj_defaults.validation_folds_filename,image_folder=proj_defaults.raw_data_folder/"imagesTr",ext=".nii.gz")



resampled_dataset_properties= load_dict(proj_defaults.resampled_dataset_properties_filename)
mask_files=[ proj_defaults.raw_data_folder/"masksTr"/test_file.name.replace(".npy",".nii.grz") for test_file in test_list]
img_files =  [proj_defaults.raw_data_folder/"imagesTr"/mask_file.name for mask_file in mask_files]

In [ ]:


run_name_w= "KITS-2284" # best trial
run_name_p  ="KITS-2302"
E = EndToEndPredictor(proj_defaults,run_name_w,run_name_p,use_neptune=True,save_localiser=True )

# Predicting all test cases in fold0 split: train,valid, test
## Note: Although each fold uses a different train / valid combination, the test cases remain the same and so never participate in model creation. So it does not matter what fold you select in getting your files

In [ ]:

results = []
BB = BoundingBoxes_to_lists()
for n in pbar(range(len(img_files))):
    E.predict(img_fn=img_files[n])
    E.score_prediction(mask_fn = mask_files[n])
    bb = E.predictor_w.get_bbox_from_pred()
    bounding_boxes = BB.encodes(bb)
    score = E.scores
    results.append({
            'case_id': E.case_id,
            'img_filename': img_files[n],
            'mask_filename': mask_files[n],
            'prediction_filename': E.predictor_p.pred_fn,
            'score0': score[0,0],
            'score1': score[1,0],
            'score2': score[2,0],

            'bounding_boxes':bounding_boxes
    })

# Storing prediction scores in file

In [ ]:

df = pd.DataFrame(results)
df.to_csv(E.output_image_folder/('results.csv'))
print("Saved results to : {}".format(E.output_image_folder/('results.csv')))

# Simple stats

In [ ]:
print("Median label1 score: {0}\nMedial label2 score: {1}\nMedial label3 score: {2}".format(df['score0'].median(), df['score1'].median(), df['score2'].median()))

# Extra: get patch_size from Bounding Box

In [ ]:

B = BBoxesToPatchSize(E.predictor_p.patch_size,E.predictor_p.sz_dest,0.1)
pp(B)